# LAB 02.03 - Pandas

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()


replicating local resources


In [2]:
from local.lib.rlxmoocapi import submit, session
session.LoginSequence(endpoint=init.endpoint, course_id=init.course_id, lab_id="L02.03", varname="student");

logging in as zamanta.estupinan@udea.edu.co... please wait

-------------
using course session ai4eng.v1.udea.r3::2025.2
success!! you are logged in
-------------


In [3]:
import numpy as np
import pandas as pd

## Task 1: Extract data

we have a dataframe of items with a price and, **sometimes**, with an extra column (`margin`).

You will have to complete a function that will **filter** the dataframe selecting the rows:

- whose price is > 100
- **OR** whose margin is >10, if the column `margin` is present

your function must return **A LIST** with the item ids of the selected rows

For instance, with the following dataframe

             price  category
    itemid                  
    56556   108.15       2.0
    73065    83.54       2.0
    36619   114.42       0.0
    73414    82.94       0.0
    13410   115.13       2.0
    66153    91.50       1.0
    77380    85.82       0.0
    73249    95.95       0.0
    11654   100.22       2.0
    11972    77.16       1.0

your must return the following list:

     [56556, 36619, 13410, 11654]
     
But if you get the following dataframe

             price  category  margin
    itemid                          
    39059    98.11       0.0   11.04
    19526    98.11       1.0   11.25
    78176    94.34       1.0   10.51
    50948   102.37       1.0   10.77
    12111    98.07       1.0    8.50
    56191    98.53       1.0   11.65
    38887    91.49       2.0   11.24
    77915   117.30       0.0    8.64
    55010    96.13       0.0    8.95
    45925    98.59       1.0   10.45
    
you must return the following list

    [39059, 19526, 78176, 50948, 56191, 38887, 77915, 45925]

In [4]:
def create_df(missing=False, n=10):
    itemid   = np.random.randint(100000, size=n)+1000
    category = np.random.randint(3, size=n)
    price    = np.round(np.random.normal(loc=100, scale=10, size=n),2)
    margin   = np.round(np.random.normal(loc=10, scale=1, size=n),2)

    if missing:
        nmissing = np.random.randint(len(price)//2)+2
        price[np.random.permutation(len(price))[:nmissing]] = np.nan

    d = pd.DataFrame(np.r_[[price, category, margin]].T, index=itemid, columns=["price", "category", "margin"])
    d.index.name="itemid"
    if np.random.random()>.5:
        d = d[d.columns[:2]]

    return d

In [5]:
d = create_df()
d

,price,category,margin
itemid,,,
83594,109.49,0.0,9.72
7744,101.45,0.0,11.50
73999,97.55,2.0,10.30
61761,95.08,1.0,9.35
5165,119.60,2.0,9.03
49395,88.08,1.0,9.68
76703,105.71,0.0,10.45
45814,92.69,0.0,11.04
47573,92.85,0.0,9.72


In [6]:
def select_items(df):
    df = df.copy()

    cond = df["price"] > 100

    if "margin" in df.columns:
        cond |= df["margin"] > 10

    return df[cond].index.astype(int).tolist()


**manually check your answer**

In [7]:
d = create_df()
d

,price,category
itemid,,
80520,108.39,1.0
72950,101.18,0.0
15690,77.76,1.0
65188,91.51,1.0
39214,97.45,1.0
72393,92.66,1.0
97101,93.21,0.0
54369,94.13,1.0
21386,113.60,2.0


In [8]:
select_items(d)

[80520, 72950, 21386, 94779]

**submit your code**

In [9]:
student.submit_task(globals(), task_id="task_01");

## Task 2: Group statistics

Complete the following function so that it returns a dataframe with the average, max and min **prices** per category.

For instance, for the following dataframe

             price  category  margin
    itemid                          
    17946    93.85       1.0   10.64
    61190    91.72       1.0    9.76
    39639   100.16       1.0   10.67
    17791   110.44       2.0    9.65
    7333    101.05       1.0    9.69
    77362   122.33       0.0   11.14
    92646   108.13       2.0   10.58
    27797    85.52       2.0   10.88
    31746    97.56       0.0    9.75
    12355   101.04       2.0    9.51
    
you should return the following dataframe

                 media  maximo  minimo
    categoria                         
    0         109.9450  122.33  97.56
    1          96.6950  101.05  91.72
    2         101.2825  110.44  85.52
    
observe that your result
- must not be a **multilevel** columnset.
- the column names and the index name must be **exactly** as in the example.
- the **index** must be of type **int**.

In [10]:
def get_stats(df):

    df = df.copy()
    stats = (
        df.groupby("category")["price"]
          .agg(["mean", "max", "min"])
    )
    stats = stats.rename(columns={
        "mean": "media",
        "max": "maximo",
        "min": "minimo"
    })
    stats.index.name = "categoria"
    stats.index = stats.index.astype(int)

    return stats


**manually check your answer**

In [11]:
d = create_df()
d

,price,category
itemid,,
31256,103.93,2.0
47289,93.14,0.0
82924,104.16,1.0
8429,91.14,0.0
10844,103.88,2.0
45240,75.55,0.0
71800,96.07,2.0
41193,116.40,0.0
5198,84.37,0.0


In [12]:
get_stats(d)

,media,maximo,minimo
categoria,,,
0,92.120000,116.40,75.55
1,102.660000,104.16,101.16
2,101.293333,103.93,96.07


**submit your code**

In [13]:
student.submit_task(globals(), task_id="task_02");

## Task 3: Fill in missing data

Fill in the missing data in the **price** column with the following procedure:

- compute the mean and std of the avaialable prices
- sample from a normal distribution with the computed mean and std (see [`np.random.normal`])(https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html), as many samples as missing data
- substitute the missing values with the samples

For instance, for this input dataframe:

             price  category  margin
    itemid                          
    18922      NaN       1.0   10.32
    69500   121.25       1.0   10.22
    76442    90.25       1.0   12.60
    33863   106.51       0.0   10.26
    15904    95.87       1.0   11.51
    41946   103.47       2.0    9.85
    85451    93.08       2.0    9.56
    70028   116.68       1.0    9.11
    26860      NaN       2.0    9.71
    12807    91.48       0.0    9.77
    
your solution might be similar to this (not exactly the same as you will be sampling data):


                 price  category  margin
    itemid                              
    18922    97.441188       1.0   10.32
    69500   121.250000       1.0   10.22
    76442    90.250000       1.0   12.60
    33863   106.510000       0.0   10.26
    15904    95.870000       1.0   11.51
    41946   103.470000       2.0    9.85
    85451    93.080000       2.0    9.56
    70028   116.680000       1.0    9.11
    26860   103.294843       2.0    9.71
    12807    91.480000       0.0    9.77
    
    
**WARN**: your function must not modify the original dataframe, make a copy of the input dataframe, fill the values in the copy and return it.

**HINT**: use the [`isna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html) method of a dataframe or a series.

In [15]:
d = create_df(missing=True)
d

,price,category,margin
itemid,,,
88563,NaN,1.0,11.24
63958,101.71,2.0,9.01
2379,NaN,1.0,9.73
69342,97.38,1.0,12.10
78110,91.65,0.0,8.89
92685,92.38,2.0,8.89
65220,NaN,2.0,9.32
36248,94.57,1.0,9.74
41893,NaN,1.0,10.21


In [16]:
def fillna(df):
    df = df.copy()
    mask = df["price"].isna()
    n_missing = mask.sum()
    if n_missing > 0:
        mean_price = df.loc[~mask, "price"].mean()
        std_price  = df.loc[~mask, "price"].std()
        samples = np.random.normal(loc=mean_price, scale=std_price, size=n_missing)
        df.loc[mask, "price"] = samples
    return df

**check manually your code**

In [17]:
d = create_df(missing=True)
d

,price,category
itemid,,
79872,108.25,2.0
64661,NaN,1.0
83734,102.03,0.0
19904,NaN,0.0
42486,112.39,1.0
88274,85.13,1.0
16865,NaN,1.0
46547,NaN,2.0
2418,101.76,0.0


In [18]:
fillna(d)

,price,category
itemid,,
79872,108.250000,2.0
64661,102.594435,1.0
83734,102.030000,0.0
19904,88.473709,0.0
42486,112.390000,1.0
88274,85.130000,1.0
16865,86.340328,1.0
46547,124.697659,2.0
2418,101.760000,0.0


**submit your code**

In [19]:
student.submit_task(globals(), task_id="task_03");